In [1]:
import glob
import numpy as np
import random
from PIL import Image
import io
import pickle

Okay. Very conveniently, all of my classes have very different sizes. This is gonna make things fun.

    Total no. of files: 1256
    100: 339
    102: 49
    104: 9
    110: 191
    112: 3
    200: 432
    201: 4
    210: 195
    300: 19
    310: 15
    
1256 does divide neatly by 157, so it would be reasonable practical to have 7 training batches plus one test batch with 157 images in them each. It's physically impossible to have every batch have an equal number of objects - there just aren't enough of some classes. A definite priority is having at least one of each class in the test set, which would only realistically affect classes 112 and 201 - all the others have enough for at least one per batch.

In [2]:
def randomise_by_index(inputlist,idx_list): # A helper function for later
 
    """
       Function to randomize an array of data
    """
     
    if len(inputlist)!=len(idx_list):
        print("These aren't the same length")
 
    outputlist = []
    for i in idx_list:
        outputlist.append(inputlist[i])
 
    return outputlist

In [3]:
# Take as an input a list of file counts (vector), no. of batches and length of batches

def multiclass_batch(class_count, batches, batch_size, overflow=True):
    
    # Set as a np array to ensure all relevant mathematial functions behave
    class_count = np.array(class_count)
    len_class = len(class_count)
    
    # First check: sum up the total number of files and make sure it equals no. x length of batches
    # If not, throw an error that it doesn't divide neatly
    if sum(class_count)==(batches*batch_size):
        
        extra_count = 0
        
    elif sum(class_count)>(batches*batch_size):
          
        # Any overflow in length should go into the test batch; this will keep track of how many should go in
        extra_count = sum(class_count) - (batches*batch_size)
        
    else:
        
        print("Error: not enough images for this many batches")
        
    # Test batch section
    
    # Divide the vector by the no. of batches and floor it
    # If any entry equals zero, set it to one instead
    test_count = np.floor(class_count/batches)
    test_count[np.where(test_count == 0)] = 1
    
    # Check if the number of test entries divides neatly by the batch size 
    if sum(test_count) == batch_size and extra_count == 0:
        
        print("Hooray! This'll be simpler!")
        # Basically, if this is the case, it makes life a lot easier and the test batch is now done
    
    # If any extras need to be added:
    elif sum(test_count) == batch_size and extra_count != 0:
        
        print("This one's still WIP")
    
    # If there's not enough entries    
    elif sum(test_count) < batch_size:
        
        # Find how many entries are left to fill, and which classes have the largest remainders when divided
        to_fill = (batch_size - sum(test_count) + extra_count).astype(int)
        remainder = class_count%batches
        
        # If there are fewer entries to be filled than there are classes
        if sum(remainder) <= len_class:
        
            # Find the indices of the largest n entries' remainders, where n is the number of entries to fill
            indices = (-remainder).argsort()[:to_fill]
            
            # Add one to the counts for each of the n largest remainders
            for i in range(to_fill):
            
                test_count[indices[i]] = test_count[indices[i]] + 1
            
        # Otherwise, there are more entries to be filled than there are classes
        # Does the number to fill divide neatly by the number of classes?
        # If so, test_count just gets that number added to every entry
        elif to_fill%len_class == 0:
        
            test_count = test_count + (to_fill/len_class)
         
        # If not, add an even number to each, then add the remainder to the largest remaining classes
        else:
            
            for i in range(int(to_fill)):
                
                test_count = test_count + np.floor(to_fill/class_count)
                
                last_fill = to_fill%len_class
                
                indices = (-remainder).argsort()[:last_fill]
                
                # Add one to the counts for each of the n largest remainders
                for i in range(last_fill):
            
                    test_count[indices[i]] = test_count[indices[i]] + 1
        
    
    # Only remaining possibility is that there's too many entries
    else:
        
        # Find out how many extra entries there are, and which classes have the largest remainders when divided
        to_remove = test_count - batch_size - extra_count
        remainder = class_count%batches
        
        # Find the indices of the smallest n entries' remainders, where n is the number of entries to fill
        indices = (remainder).argsort()[:to_fill]
        
        # Prep for the while loop
        no_replaced = 0
        
        # Check if any of the chosen classes have only one image
        while any (test_count[indices] == 1) is True:
            
            # If they do, find how many
            to_replace = np.where(test_count[indices]) == 1
            no_to_replace = len(to_replace)
            
            # Find the next largest remainders
            index_replacements = (remainder).argsort()[to_fill+no_replaced:to_fill+no_replaced+no_to_replace]
            
            # Replace the previous entries
            for i in range(no_to_replace):
                
                indices[to_replace[i]] = index_replacements[i]
            
            # Note how many have now been replaced in case another pass is needed
            no_replaced = no_replaced + no_to_replaced
        
        # Now all the checks have been done, should be free to remove all necessary
        for i in range(to_fill):
            
            test_count[indices[i]] = test_count[indices[i]] - 1
 
        
    # Training batch section
    
    # Find the number of remaining entries
    training_count = class_count - test_count
    
    # Total no. of images of each class present in every batch:
    base_count = np.floor(training_count/(batches-1))
    
    # Number of remaining images of each class
    extra_count = training_count%(batches-1)
    
    # Check if the base count already contains batch_size images
    if sum(base_count) == batch_size:
        
        print("Hooray! That'll make life easier")
        # This makes life easier! It's now done.
        
        training_count = np.array([base_count,]*(batches-1))
        
    else:
            
        # Find the number to add to each batch
        to_add = batch_size - sum(base_count)
            
        # Repeat the base_count vector (batches-1) times
        training_count = np.array([base_count,]*(batches-1))
        
        # For every row, to_add times
        for i in range(batches-1):
            
            for j in range(to_add.astype(int)):
                
                # Check which (if any) of the classes have images to use
                index_options = np.asarray(np.nonzero(extra_count)[0])
                
                # Select a random one of these indices
                index_choice = random.choice(index_options)
                
                # Add one to training_count in relevant entry; subtract one from extra_count in relevant entry
                training_count[i, index_choice] = training_count[i, index_choice] + 1
                extra_count[index_choice] = extra_count[index_choice] - 1
    
    output = np.vstack((training_count, test_count))
    
    return output

In [4]:
# Directory to save batches in
binddir = './MiraBest_basic_batches/'

# Training plus test batches
nbatch = 10 # Edited for MiraBest 21/02/20

# No. of images per batch
pbatch = 77 # Edited for MiraBest 21/02/20

# Label names
label_names = ['100', '200']

# Length of data arrays [npix x npix x greyscale = 150 x 150 x 1 = 22500]
nvis = 22500

# Generate the lists of each class and randomly shuffle the order
cl0_files = np.array(glob.glob('./MiraBest_data/100*.png'))
cl1_files = np.array(glob.glob('./MiraBest_data/200*.png'))


# Count no. of objects in each class
n_cl0 = len(cl0_files)
n_cl1 = len(cl1_files)


# Ensure there are enough objects
assert ((n_cl0+n_cl1)>=(pbatch*nbatch)),'Not enough samples available to fill '+str(nbatch)+' batches of '+str(pbatch)


# WIP from here
 
# No. of images in each class    
class_count = np.asarray([339, 432])

# Generate some matrices of how many of each class should go in each batch
batch_count = multiclass_batch(class_count, 10, 77)

# For tracking how many images have been used
count = np.zeros(2)
    
# Loop through and fill the batches:
 
for batch in range(nbatch):
 
    if (batch==(nbatch-1)):
        # the last batch is the test batch:
        oname = "test_batch"
        batch_label = 'testing batch 1 of 1'
    else:
        # everything else is a training batch:
        oname = "data_batch_"+str(batch+1)
        batch_label = 'training batch '+str(batch+1)+' of '+str(nbatch-1)
 
    # create empty arrays for the batches:
    labels=[];filedata=[];data=[];filenames=[]
 
    # Class 0
    for i in range(int(count[0]),int(count[0]+batch_count[batch,0])):
 
        filename = cl0_files[i]
        filenames.append(filename)
 
        labels.append(0)
 
        im = Image.open(filename)
        im = (np.array(im))
        filedata = np.array(list(im), np.uint8)
        data.append(filedata)
        
    count[0] = count[0] + batch_count[batch,0]
 
    # Class 1
    for i in range(int(count[1]),int(count[1]+batch_count[batch,1])):
 
        filename = cl1_files[i]
        filenames.append(filename)
 
        labels.append(1)
 
        im = Image.open(filename)
        im = (np.array(im))
        filedata = np.array(list(im), np.uint8)
        data.append(filedata)
        
    count[1] = count[1] + batch_count[batch,1]
    
    # randomise data in batch:
    idx_list = range(0,pbatch)
    labels = randomise_by_index(labels,idx_list)
    data = randomise_by_index(data,idx_list)
    filenames = randomise_by_index(filenames,idx_list)
 
    # create dictionary of batch:
    dict = {
            'batch_label':batch_label,
            'labels':labels,
            'data':data,
            'filenames':filenames
            }
 
    # write pickled output:
    with io.open(binddir+oname, 'wb') as f:
        pickle.dump(dict, f)
 
    # end batch loop

These aren't the same length
These aren't the same length
These aren't the same length


In [5]:
# total number of samples per batch: [modified 20190925]
pbatch = 77

# label names: [modified 20190415]
label_names = ['100', '200']

# length of data arrays [npix x npix x rgb = 150 x 150 x 1 = 22500] [modified 20190415]
nvis = 22500

# ------------------------------------
# now write the meta data file:
oname = 'batches.meta'

# ------------------------------------
# create dictionary of batch:
dict = {
        'num_cases_per_batch':pbatch,
        'label_names':label_names,
        'num_vis':nvis,
        }

# ------------------------------------
# write pickled output:
with io.open(binddir+oname, 'wb') as f:
    pickle.dump(dict, f)

In [4]:
# Directory to save batches in
binddir = './MiraBest_batches/'

# Training plus test batches
nbatch = 8 # Edited for MiraBest 21/02/20

# No. of images per batch
pbatch = 157 # Edited for MiraBest 21/02/20

# Label names
label_names = ['100', '102', '104', '110', '112', '200', '201', '210', '300', '310']

# Length of data arrays [npix x npix x greyscale = 150 x 150 x 1 = 22500]
nvis = 22500

# Generate the lists of each class and randomly shuffle the order
cl0_files = np.array(glob.glob('./MiraBest_data/100*.png'))
cl1_files = np.array(glob.glob('./MiraBest_data/102*.png'))
cl2_files = np.array(glob.glob('./MiraBest_data/104*.png'))
cl3_files = np.array(glob.glob('./MiraBest_data/110*.png'))
cl4_files = np.array(glob.glob('./MiraBest_data/112*.png'))
cl5_files = np.array(glob.glob('./MiraBest_data/200*.png'))
cl6_files = np.array(glob.glob('./MiraBest_data/201*.png'))
cl7_files = np.array(glob.glob('./MiraBest_data/210*.png'))
cl8_files = np.array(glob.glob('./MiraBest_data/300*.png'))
cl9_files = np.array(glob.glob('./MiraBest_data/310*.png'))

# Count no. of objects in each class
n_cl0 = len(cl0_files)
n_cl1 = len(cl1_files)
n_cl2 = len(cl2_files)
n_cl3 = len(cl3_files)
n_cl4 = len(cl4_files)
n_cl5 = len(cl5_files)
n_cl6 = len(cl6_files)
n_cl7 = len(cl7_files)
n_cl8 = len(cl8_files)
n_cl9 = len(cl9_files)

# Ensure there are enough objects
assert ((n_cl0+n_cl1+n_cl2+n_cl3+n_cl4+n_cl5+n_cl6+n_cl7+n_cl8+n_cl9)>=(pbatch*nbatch)),'Not enough samples available to fill '+str(nbatch)+' batches of '+str(pbatch)


# WIP from here
 
# No. of images in each class    
class_count = np.asarray([339, 49, 9, 191, 3, 432, 4, 195, 19, 15])

# Generate some matrices of how many of each class should go in each batch
batch_count = multiclass_batch(class_count, 8, 157)

# For tracking how many images have been used
count = np.zeros(10)
    
# Loop through and fill the batches:
 
for batch in range(nbatch):
 
    if (batch==(nbatch-1)):
        # the last batch is the test batch:
        oname = "test_batch"
        batch_label = 'testing batch 1 of 1'
    else:
        # everything else is a training batch:
        oname = "data_batch_"+str(batch+1)
        batch_label = 'training batch '+str(batch+1)+' of '+str(nbatch-1)
 
    # create empty arrays for the batches:
    labels=[];filedata=[];data=[];filenames=[]
 
    # Class 0
    for i in range(int(count[0]),int(count[0]+batch_count[batch,0])):
 
        filename = cl0_files[i]
        filenames.append(filename)
 
        labels.append(0)
 
        im = Image.open(filename)
        im = (np.array(im))
        filedata = np.array(list(im), np.uint8)
        data.append(filedata)
        
    count[0] = count[0] + batch_count[batch,0]
 
    # Class 1
    for i in range(int(count[1]),int(count[1]+batch_count[batch,1])):
 
        filename = cl1_files[i]
        filenames.append(filename)
 
        labels.append(1)
 
        im = Image.open(filename)
        im = (np.array(im))
        filedata = np.array(list(im), np.uint8)
        data.append(filedata)
        
    count[1] = count[1] + batch_count[batch,1]
        
    # Class 2
    for i in range(int(count[2]),int(count[2]+batch_count[batch,2])):
 
        filename = cl2_files[i]
        filenames.append(filename)
 
        labels.append(2)
 
        im = Image.open(filename)
        im = (np.array(im))
        filedata = np.array(list(im), np.uint8)
        data.append(filedata)
        
    count[2] = count[2] + batch_count[batch,2]
    
    # Class 3
    for i in range(int(count[3]),int(count[3]+batch_count[batch,3])):
 
        filename = cl3_files[i]
        filenames.append(filename)
 
        labels.append(3)
 
        im = Image.open(filename)
        im = (np.array(im))
        filedata = np.array(list(im), np.uint8)
        data.append(filedata)
        
    count[3] = count[3] + batch_count[batch,3]
    
    # Class 4
    if batch_count[batch,4] != 0:
    
        for i in range(int(count[4]),int(count[4]+batch_count[batch,4])):

            filename = cl4_files[i]
            filenames.append(filename)

            labels.append(4)

            im = Image.open(filename)
            im = (np.array(im))
            filedata = np.array(list(im), np.uint8)
            data.append(filedata)

        count[4] = count[4] + batch_count[batch,4]
        
    # Class 5
    for i in range(int(count[5]),int(count[5]+batch_count[batch,5])):
 
        filename = cl5_files[i]
        filenames.append(filename)
 
        labels.append(5)
 
        im = Image.open(filename)
        im = (np.array(im))
        filedata = np.array(list(im), np.uint8)
        data.append(filedata)
        
    count[5] = count[5] + batch_count[batch,5]
        
    # Class 6
    if batch_count[batch,6] != 0:
    
        for i in range(int(count[6]),int(count[6]+batch_count[batch,6])):

            filename = cl6_files[i]
            filenames.append(filename)

            labels.append(6)

            im = Image.open(filename)
            im = (np.array(im))
            filedata = np.array(list(im), np.uint8)
            data.append(filedata)

        count[6] = count[6] + batch_count[batch,6]
        
    # Class 7
    for i in range(int(count[7]),int(count[7]+batch_count[batch,7])):
 
        filename = cl7_files[i]
        filenames.append(filename)
 
        labels.append(7)
 
        im = Image.open(filename)
        im = (np.array(im))
        filedata = np.array(list(im), np.uint8)
        data.append(filedata)
        
    count[7] = count[7] + batch_count[batch,7]
    
    # Class 8
    for i in range(int(count[8]),int(count[8]+batch_count[batch,8])):
 
        filename = cl8_files[i]
        filenames.append(filename)
 
        labels.append(8)
 
        im = Image.open(filename)
        im = (np.array(im))
        filedata = np.array(list(im), np.uint8)
        data.append(filedata)
        
    count[8] = count[8] + batch_count[batch,8]
        
    # Class 9
    for i in range(int(count[9]),int(count[9]+batch_count[batch,9])):
 
        filename = cl9_files[i]
        filenames.append(filename)
 
        labels.append(9)
 
        im = Image.open(filename)
        im = (np.array(im))
        filedata = np.array(list(im), np.uint8)
        data.append(filedata)
        
    count[9] = count[9] + batch_count[batch,9]
    
    
    # randomise data in batch:
    idx_list = range(0,pbatch)
    labels = randomise_by_index(labels,idx_list)
    data = randomise_by_index(data,idx_list)
    filenames = randomise_by_index(filenames,idx_list)
 
    # create dictionary of batch:
    dict = {
            'batch_label':batch_label,
            'labels':labels,
            'data':data,
            'filenames':filenames
            }
 
    # write pickled output:
    with io.open(binddir+oname, 'wb') as f:
        pickle.dump(dict, f)
 
    # end batch loop

In [5]:
# total number of samples per batch: [modified 20190925]
pbatch = 157

# label names: [modified 20190415]
label_names = ['100', '102', '104', '110', '112', '200', '201', '210', '300', '310']

# length of data arrays [npix x npix x rgb = 150 x 150 x 1 = 22500] [modified 20190415]
nvis = 22500

# ------------------------------------
# now write the meta data file:
oname = 'batches.meta'

# ------------------------------------
# create dictionary of batch:
dict = {
        'num_cases_per_batch':pbatch,
        'label_names':label_names,
        'num_vis':nvis,
        }

# ------------------------------------
# write pickled output:
with io.open(binddir+oname, 'wb') as f:
    pickle.dump(dict, f)

In [2]:
#01/12/20 - and now for MiraBest-NVSS

with open('./testset_files.txt') as f:
    testbatch = [line.rstrip() for line in f]
    
with open('./trainingbatch_files.txt') as f:
    trainingbatch = [line.rstrip() for line in f]

In [3]:
def label_batches(file):

    labels = np.zeros(len(file))
    
    for i in range(len(file)):
    
        if file[i][0:3] == '100':
        
            labels[1] = 0
        
        elif file[i][0:3] == '102':
        
            labels[i] = 1
        
        elif file[i][0:3] == '104':
        
            labels[i] = 2
        
        elif file[i][0:3] == '110':
        
            labels[i] = 3
        
        elif file[i][0:3] == '112':
        
            labels[i] = 4
        
        elif file[i][0:3] == '200':
        
            labels[i] = 5
        
        elif file[i][0:3] == '201':
        
            labels[i] = 6
        
        elif file[i][0:3] == '210':
        
            labels[i] = 7
        
        elif file[i][0:3] == '300':
        
            labels[i] = 8
        
        elif file[i][0:3] == '310':
        
            labels[i] = 9
            
    return labels

In [4]:
import numpy as np

testlabels = label_batches(testbatch)
traininglabels = label_batches(trainingbatch)

In [6]:
print(len(traininglabels), len(testlabels))

1099 157


In [8]:
# Directory to save batches in
binddir = './MiraBestN/batches/'

nbatch = 8 # Edited for MiraBest 21/02/20

pbatch = 157 # Edited for MiraBest 21/02/20

pathstart = './MiraBestN/PNG/Scaled_Final/'

for batch in range(nbatch):
    
    if (batch==(nbatch-1)):
        # the last batch is the test batch:
        oname = "test_batch"
        batch_label = 'testing batch 1 of 1'
    else:
        # everything else is a training batch:
        oname = "data_batch_"+str(batch+1)
        batch_label = 'training batch '+str(batch+1)+' of '+str(nbatch-1)
 
    # create empty arrays for the batches:
    labels=[];filedata=[];data=[];filenames=[]
    
    if batch < 7:
        
        for i in range(157):
        
            print(i, batch, i+157*batch)
            filename = pathstart+trainingbatch[i+157*batch]
            filenames.append(filename)
 
            labels.append(traininglabels[i+157*batch])
 
            im = Image.open(filename)
            im = (np.array(im))
            filedata = np.array(list(im), np.uint8)
            data.append(filedata)
            
        # create dictionary of batch:
        dict = {
                'batch_label':batch_label,
                'labels':labels,
                'data':data,
                'filenames':filenames
                }
 
        # write pickled output:
        with io.open(binddir+oname, 'wb') as f:
            pickle.dump(dict, f)
            
    else:
        
        for i in range(157):
        
            filename = pathstart+testbatch[i]
            filenames.append(filename)
 
            labels.append(testlabels[i])
 
            im = Image.open(filename)
            im = (np.array(im))
            filedata = np.array(list(im), np.uint8)
            data.append(filedata)
            
        # create dictionary of batch:
        dict = {
                'batch_label':batch_label,
                'labels':labels,
                'data':data,
                'filenames':filenames
                }
 
        # write pickled output:
        with io.open(binddir+oname, 'wb') as f:
            pickle.dump(dict, f)

0 0 0
1 0 1
2 0 2
3 0 3
4 0 4
5 0 5
6 0 6
7 0 7
8 0 8
9 0 9
10 0 10
11 0 11
12 0 12
13 0 13
14 0 14
15 0 15
16 0 16
17 0 17
18 0 18
19 0 19
20 0 20
21 0 21
22 0 22
23 0 23
24 0 24
25 0 25
26 0 26
27 0 27
28 0 28
29 0 29
30 0 30
31 0 31
32 0 32
33 0 33
34 0 34
35 0 35
36 0 36
37 0 37
38 0 38
39 0 39
40 0 40
41 0 41
42 0 42
43 0 43
44 0 44
45 0 45
46 0 46
47 0 47
48 0 48
49 0 49
50 0 50
51 0 51
52 0 52
53 0 53
54 0 54
55 0 55
56 0 56
57 0 57
58 0 58
59 0 59
60 0 60
61 0 61
62 0 62
63 0 63
64 0 64
65 0 65
66 0 66
67 0 67
68 0 68
69 0 69
70 0 70
71 0 71
72 0 72
73 0 73
74 0 74
75 0 75
76 0 76
77 0 77
78 0 78
79 0 79
80 0 80
81 0 81
82 0 82
83 0 83
84 0 84
85 0 85
86 0 86
87 0 87
88 0 88
89 0 89
90 0 90
91 0 91
92 0 92
93 0 93
94 0 94
95 0 95
96 0 96
97 0 97
98 0 98
99 0 99
100 0 100
101 0 101
102 0 102
103 0 103
104 0 104
105 0 105
106 0 106
107 0 107
108 0 108
109 0 109
110 0 110
111 0 111
112 0 112
113 0 113
114 0 114
115 0 115
116 0 116
117 0 117
118 0 118
119 0 119
120 0 120
121 0 121


131 5 916
132 5 917
133 5 918
134 5 919
135 5 920
136 5 921
137 5 922
138 5 923
139 5 924
140 5 925
141 5 926
142 5 927
143 5 928
144 5 929
145 5 930
146 5 931
147 5 932
148 5 933
149 5 934
150 5 935
151 5 936
152 5 937
153 5 938
154 5 939
155 5 940
156 5 941
0 6 942
1 6 943
2 6 944
3 6 945
4 6 946
5 6 947
6 6 948
7 6 949
8 6 950
9 6 951
10 6 952
11 6 953
12 6 954
13 6 955
14 6 956
15 6 957
16 6 958
17 6 959
18 6 960
19 6 961
20 6 962
21 6 963
22 6 964
23 6 965
24 6 966
25 6 967
26 6 968
27 6 969
28 6 970
29 6 971
30 6 972
31 6 973
32 6 974
33 6 975
34 6 976
35 6 977
36 6 978
37 6 979
38 6 980
39 6 981
40 6 982
41 6 983
42 6 984
43 6 985
44 6 986
45 6 987
46 6 988
47 6 989
48 6 990
49 6 991
50 6 992
51 6 993
52 6 994
53 6 995
54 6 996
55 6 997
56 6 998
57 6 999
58 6 1000
59 6 1001
60 6 1002
61 6 1003
62 6 1004
63 6 1005
64 6 1006
65 6 1007
66 6 1008
67 6 1009
68 6 1010
69 6 1011
70 6 1012
71 6 1013
72 6 1014
73 6 1015
74 6 1016
75 6 1017
76 6 1018
77 6 1019
78 6 1020
79 6 1021
80 6 102

FileNotFoundError: [Errno 2] No such file or directory: './MiraBestN/PNG/Scaled_Final/200_149.170-000.023_0.1392_0107.66.png'

In [63]:
print(len(trainingbatch))

1099


In [2]:
import numpy as np
import glob

cl0_files = np.array(glob.glob('./MiraBestN/PNG/Scaled_Final/100*.png'))
cl1_files = np.array(glob.glob('./MiraBestN/PNG/Scaled_Final/102*.png'))
cl2_files = np.array(glob.glob('./MiraBestN/PNG/Scaled_Final/104*.png'))
cl3_files = np.array(glob.glob('./MiraBestN/PNG/Scaled_Final/110*.png'))
cl4_files = np.array(glob.glob('./MiraBestN/PNG/Scaled_Final/112*.png'))
cl5_files = np.array(glob.glob('./MiraBestN/PNG/Scaled_Final/200*.png'))
cl6_files = np.array(glob.glob('./MiraBestN/PNG/Scaled_Final/201*.png'))
cl7_files = np.array(glob.glob('./MiraBestN/PNG/Scaled_Final/210*.png'))
cl8_files = np.array(glob.glob('./MiraBestN/PNG/Scaled_Final/300*.png'))
cl9_files = np.array(glob.glob('./MiraBestN/PNG/Scaled_Final/310*.png'))